In [5]:
!pip install datasets
!pip install transformers

distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None


In [6]:
from datasets import load_dataset
dataset = load_dataset('squad_kor_v1')

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


In [8]:
from transformers import AutoTokenizer
import numpy as np

model_checkpoint = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [9]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [11]:
tokenized_input = tokenizer(dataset['train'][0]['context'],padding='max_length',truncation=True)

In [13]:
tokenizer.decode(tokenized_input['input_ids'])

'[CLS] 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 [UNK] 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [17]:
from tqdm import tqdm, trange
import argparse
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

torch.manual_seed(2021)
torch.cuda.manual_seed(2021)
np.random.seed(2021)
random.seed(2021)

In [19]:
sample_idx = np.random.choice(range(len(dataset['train'])),128)

training_dataset = dataset['train'][sample_idx]

print(len(dataset['train']),len(training_dataset))

60407 5


In [22]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)
q_seqs = tokenizer(training_dataset['question'],padding='max_length',truncation=True,return_tensors='pt')
p_seqs = tokenizer(training_dataset['context'],padding='max_length',truncation=True,return_tensors='pt')

In [23]:
train_dataset = TensorDataset(p_seqs['input_ids'],p_seqs['attention_mask'],p_seqs['token_type_ids'],
                             q_seqs['input_ids'],q_seqs['attention_mask'],q_seqs['token_type_ids'])

In [41]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self,config):
        super(BertEncoder,self).__init__(config)
        
        self.bert = BertModel(config)
        self.init_weights()
        
    def forward(self,input_ids,attention_mask=None,token_type_ids=None):
        outputs = self.bert(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        return pooled_output


In [42]:
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
    p_encoder.cuda()
    q_encoder.cuda()
    print('GPU enabled')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias',

GPU enabled


In [43]:
def train(args, dataset, p_model, q_model):
    # DataLoader
    train_sampler = RandomSampler(dataset)
    train_dataloader = DataLoader(dataset, sampler = train_sampler, batch_size= args.per_device_train_batch_size)
    
    #Optimizer
    
    no_decay = ['bias','LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params':[p for n,p in p_model.named_parameters() if not any(nd in n for nd in no_decay)],'weight_decay':args.weight_decay},
        {'params':[p for n,p in p_model.named_parameters() if any(nd in n for nd in no_decay)],'weight_decay':0.0},
        {'params':[p for n,p in q_model.named_parameters() if not any(nd in n for nd in no_decay)],'weight_decay':args.weight_decay},
        {'params':[p for n,p in q_model.named_parameters() if any(nd in n for nd in no_decay)],'weight_decay':0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters,lr = args.learning_rate, eps=args.adam_epsilon)
    t_total = len(train_dataloader) // args.gradient_accumulation_steps*args.num_train_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=args.warmup_steps,num_training_steps=t_total)
    
    #Bert training
    global_step =0
    p_model.zero_grad()
    q_model.zero_grad()
    
    torch.cuda.empty_cache()
    
    train_iterator = trange(int(args.num_train_epochs),desc='Epoch')
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader,desc ='Iteration')
        for step, batch in enumerate(epoch_iterator):
            q_encoder.train()
            p_encoder.train()
            
            if torch.cuda.is_available():
                batch = tuple(t.cuda() for t in batch)
                
            p_inputs = {'input_ids':batch[0],
                       'attention_mask':batch[1],
                       'token_type_ids':batch[2]}
            q_inputs = {'input_ids':batch[3],
                       'attention_mask':batch[4],
                       'token_type_ids':batch[5]}
            
            p_outputs = p_model(**p_inputs) #(batch_size,emb_dim)
            q_outputs = q_model(**q_inputs) #(batch_size,emb_dim)
            
            #Calculate similarity score & loss
            
            sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs,0,1))
            
            # target
            
            targets = torch.arange(0,args.per_device_train_batch_size).long()
            if torch.cuda.is_available():
                targets=targets.to('cuda')
                
            sim_scores = F.log_softmax(sim_scores,dim=1)
            loss = F.nll_loss(sim_scores,targets)
            print(loss)
            loss.backward()
            optimizer.step()
            scheduler.step()
            q_model.zero_grad()
            p_model.zero_grad()
            global_step += 1
            
            torch.cuda.empty_cache()
            
        return p_model,q_model

In [44]:
args = TrainingArguments(
        output_dir='dense_retireval',
        evaluation_strategy='epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=2,
        weight_decay=0.01
)

In [45]:
p_encoder,q_encoder = train(args,train_dataset,p_encoder,q_encoder)











Epoch:   0%|          | 0/2 [00:00<?, ?it/s]










Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

tensor(30.8067, device='cuda:0', grad_fn=<NllLossBackward>)













Iteration:   3%|▎         | 1/32 [00:00<00:15,  2.04it/s]

tensor(3.7890, device='cuda:0', grad_fn=<NllLossBackward>)













Iteration:   6%|▋         | 2/32 [00:00<00:14,  2.13it/s]

...












Iteration:  94%|█████████▍| 30/32 [00:13<00:00,  2.18it/s]

tensor(1.2258, device='cuda:0', grad_fn=<NllLossBackward>)













Iteration:  97%|█████████▋| 31/32 [00:14<00:00,  2.17it/s]

tensor(0.9605, device='cuda:0', grad_fn=<NllLossBackward>)













Iteration: 100%|██████████| 32/32 [00:14<00:00,  2.19it/s]


In [47]:
valid_corpus = list(set(example['context'] for example in dataset['validation']))[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']
print(query,ground_truth)

유아인에게 타고난 배우라고 말한 드라마 밀회의 감독은? 화보 촬영을 위해 미국에 있을 때, 김희애의 연락을 통해 JTBC 드라마 《밀회》의 캐스팅을 제안받았다. 당시 영화 《베테랑》에 이미 캐스팅된 상태였으나, 유아인은 류승완 감독과 제작사의 양해를 얻어 《밀회》에 출연한다. 천재 피아니스트 ‘이선재’ 역할을 위해 피아니스트들의 영상을 보고 곡의 스피드와 건반 위치 등을 외워 실제 타건을 하며 촬영했다. 피아노 울림판을 수건으로 막고 타건을 하면, 그 후 대역 피아니스트의 소리를 덧입히는 방식이었다. 《밀회》는 작품성을 인정받고 숱한 화제를 낳으며 당시 종편으로서는 높은 시청률을 기록했다. 유아인은 섬세한 연기력을 선보여 순수함으로 시청자들을 매료시켰다는 호평을 얻었고, 특히 피아노 연주에 있어서 클래식 종사자들에게 인정을 받았다. 연출을 맡은 안판석 감독은 유아인에 대해 “느낌으로만 연기를 하는 게 아니고 감성을 지적으로 통제해 가면서 연기한다. 그 나이에”라며 “타고난 배우”라고 말했다. 유아인은 《밀회》를 통해 예술적인 면모를 구체화할 수 있어서 만족감을 느꼈다고 밝혔으며, 종영 후 자신의 페이스북 계정에 긴 소감글을 남겼다. 특히 ‘이선재’ 캐릭터를 배우 유아인이 가진 소년성의 엑기스로 생각하며, 2015년 10월 부산국제영화제 오픈토크에서는 본인이 가장 좋아하는 캐릭터로 꼽았다.


In [48]:
if not ground_truth in valid_corpus:
    valid_corpus.append(ground_truth)


In [50]:
def to_cuda(batch):
    return tuple(t.cuda() for t in batch)

In [55]:
with torch.no_grad():
    p_encoder.eval()
    q_encoder.eval()
    
    q_seqs_val = tokenizer([query],padding='max_length',truncation=True,return_tensors='pt').to('cuda')
    q_emb = q_encoder(**q_seqs_val).to('cpu')
    p_embs = []
    for p in valid_corpus:
        p = tokenizer(p,padding='max_length',truncation=True,return_tensors='pt').to('cuda')
        p_emb = p_encoder(**p).to('cpu').numpy()
        p_embs.append(p_emb)
        
    p_embs = torch.Tensor(p_embs).squeeze()
    print(p_embs.size(),q_emb.size())

torch.Size([11, 768]) torch.Size([1, 768])


In [56]:
dot_prod_scores = torch.matmul(q_emb,torch.transpose(p_embs,0,1))

In [58]:
print(dot_prod_scores.size())

torch.Size([1, 11])


In [60]:
rank= torch.argsort(dot_prod_scores,dim=1,descending=True).squeeze()
print(dot_prod_scores)
print(rank)

tensor([[29.4267, 31.4995, 31.0922, 30.9991, 29.3170, 32.3554, 31.3094, 32.7454,
         32.6979, 30.2796, 32.1270]])
tensor([ 7,  8,  5, 10,  1,  6,  2,  3,  9,  0,  4])


In [63]:
k = 11
print('Search query',query,'\n')
print('ground truth passage')
print(ground_truth,'\n')

for i in range(k):
    print("top-%d passage with score %.4f" % (i+1,dot_prod_scores.squeeze()[rank[i]]))
    print(valid_corpus[rank[i]])

Search query 유아인에게 타고난 배우라고 말한 드라마 밀회의 감독은? 

ground truth passage
화보 촬영을 위해 미국에 있을 때, 김희애의 연락을 통해 JTBC 드라마 《밀회》의 캐스팅을 제안받았다. 당시 영화 《베테랑》에 이미 캐스팅된 상태였으나, 유아인은 류승완 감독과 제작사의 양해를 얻어 《밀회》에 출연한다. 천재 피아니스트 ‘이선재’ 역할을 위해 피아니스트들의 영상을 보고 곡의 스피드와 건반 위치 등을 외워 실제 타건을 하며 촬영했다. 피아노 울림판을 수건으로 막고 타건을 하면, 그 후 대역 피아니스트의 소리를 덧입히는 방식이었다. 《밀회》는 작품성을 인정받고 숱한 화제를 낳으며 당시 종편으로서는 높은 시청률을 기록했다. 유아인은 섬세한 연기력을 선보여 순수함으로 시청자들을 매료시켰다는 호평을 얻었고, 특히 피아노 연주에 있어서 클래식 종사자들에게 인정을 받았다. 연출을 맡은 안판석 감독은 유아인에 대해 “느낌으로만 연기를 하는 게 아니고 감성을 지적으로 통제해 가면서 연기한다. 그 나이에”라며 “타고난 배우”라고 말했다. 유아인은 《밀회》를 통해 예술적인 면모를 구체화할 수 있어서 만족감을 느꼈다고 밝혔으며, 종영 후 자신의 페이스북 계정에 긴 소감글을 남겼다. 특히 ‘이선재’ 캐릭터를 배우 유아인이 가진 소년성의 엑기스로 생각하며, 2015년 10월 부산국제영화제 오픈토크에서는 본인이 가장 좋아하는 캐릭터로 꼽았다. 

top-1 passage with score 32.7454
이회창 후보의 친동생 이회성과 이 후보의 최측근 서정우 변호사 등으로 구성된 대선 비선조직인 부국팀이 병역문제로 대선자금이 잘 걷히지를 않자 국세청을 동원한 대선자금 모집을 이회창 후보에게 보고한후 1997년 10월 하순 부터 대선직전까지 국세청 차장 이석희와 선거 자금을 불법으로 모은 사건. 부국팀은 9월에 차수명 한나라당 재정위원장으로부터 기탁금 고액미납자 명단을 건네받아 미납기업들을 상대로 기탁금 납부를 독촉하였는데 이때 국세청과 한나라당이 대선